## Byte-Pair Encoding
Recall: The purpose of tokenization is to break up a section of text into repeated, manageable pieces with a relatively consistent meaning
* root words, prefixes, suffixes, etc.

Last class, we talked about Byte-Pair Encoding, which repeatedly goes through a sample of text, merging the most common pairs of characters/tokens

In [26]:
# setting up an input string
word_counts = [("hug", 5), ("pug", 3), ("pun", 6), ("bun", 2), ("hugs", 3)]
input_text = ""
for word, count in word_counts:
    input_text += f"{word} " * count

input_text = input_text[:-1] # remove trailing whitespace

print(input_text)

hug hug hug hug hug pug pug pug pun pun pun pun pun pun bun bun hugs hugs hugs


In [27]:
from bytepairencoding import BytePairEncoder
from pretoken import Pretoken

bpe = BytePairEncoder()

bpe.train(input_text, 14)
tokenized_text = bpe.tokenize(input_text)

print(tokenized_text)

['hug', 'Ġhug', 'Ġhug', 'Ġhug', 'Ġhug', 'Ġp', 'ug', 'Ġp', 'ug', 'Ġp', 'ug', 'Ġpun', 'Ġpun', 'Ġpun', 'Ġpun', 'Ġpun', 'Ġpun', 'Ġ', 'b', 'un', 'Ġ', 'b', 'un', 'Ġhug', 's', 'Ġhug', 's', 'Ġhug', 's']


## How does it work?
Recall from last class: 
* First, we split the text into pretokens (often on whitespace and punctuation)
* then, we split those pretokens into characters
* finally, we merge consecutive tokens according to our vocabulary


In [3]:
vocabulary = ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]
pre_tokens = ["pun","bug","hugs","mug"]
split_pre_tokens = [["p","u","n"],["b","u","g"],["h","u","g","s"],["m","u","g"]]
split_pre_tokens = [["p","u","n"],["b","u","g"],["h","u","g","s"],["[UNK]","u","g"]] #there is no m in the vocabulary
after_first_merge_rule = [["p","u","n"],["b","ug"],["h","ug","s"],["[UNK]","ug"]]
after_second_merge_rule = [["p","un"],["b","ug"],["h","ug","s"],["[UNK]","ug"]]
after_third_merge_rule = [["p","un"],["b","ug"],["hug","s"],["[UNK]","ug"]] 
final_tokens = ["p","un","b","ug","hug","s","[UNK]","ug"]

Pretokenizer iterates through the input text, collapsing whitespace and converting punctuation to a standalone token. 

In [28]:
# see:
BytePairEncoder._pretokenize

print(bpe._pretokenize(input_text))

['h -> u -> g', 'Ġ -> h -> u -> g', 'Ġ -> h -> u -> g', 'Ġ -> h -> u -> g', 'Ġ -> h -> u -> g', 'Ġ -> p -> u -> g', 'Ġ -> p -> u -> g', 'Ġ -> p -> u -> g', 'Ġ -> p -> u -> n', 'Ġ -> p -> u -> n', 'Ġ -> p -> u -> n', 'Ġ -> p -> u -> n', 'Ġ -> p -> u -> n', 'Ġ -> p -> u -> n', 'Ġ -> b -> u -> n', 'Ġ -> b -> u -> n', 'Ġ -> h -> u -> g -> s', 'Ġ -> h -> u -> g -> s', 'Ġ -> h -> u -> g -> s']


Then, we iterate through the pretokens applying our merge rules (vocabulary). 

One key difference here is that the pretokens aren't actually strings - think of them more like linked lists; when applying a merge rule, we're really just combining two consecutive nodes into one. 

In [29]:
# see: 
BytePairEncoder.tokenize
Pretoken.apply_merge_rule

print("vocabulary:")
print(bpe.vocabulary)

print("tokenized text:")
print(tokenized_text)

vocabulary:
['s', 'n', 'h', 'Ġ', 'b', 'u', 'p', 'g', 'ug', 'Ġp', 'un', 'hug', 'Ġhug', 'Ġpun']
tokenized text:
['hug', 'Ġhug', 'Ġhug', 'Ġhug', 'Ġhug', 'Ġp', 'ug', 'Ġp', 'ug', 'Ġp', 'ug', 'Ġpun', 'Ġpun', 'Ġpun', 'Ġpun', 'Ġpun', 'Ġpun', 'Ġ', 'b', 'un', 'Ġ', 'b', 'un', 'Ġhug', 's', 'Ġhug', 's', 'Ġhug', 's']


## What about training?
First, we generate a corpus of tokens and their frequencies and set the initial vocabulary of unique characters. 

Then, we iterate through the list of tokens, finding the most common pair of tokens, merging them, and adding to the vocabulary. Rinse and repeat until we either a) hit the vocabulary limit; or b) run out of tokens to merge. 

In [6]:
# see: 
BytePairEncoder.train

pt = bpe._pretokenize(input_text)
print(bpe._generate_token_corpus(pt))

[('Ġ -> p -> u -> n', 6), ('Ġ -> h -> u -> g', 4), ('Ġ -> p -> u -> g', 3), ('Ġ -> h -> u -> g -> s', 3), ('Ġ -> b -> u -> n', 2), ('h -> u -> g', 1)]


## Testing on a larger sample of text
Here we'll fetch the text of Sherlock Holmes we used towards the end of class. 


In [30]:
import requests

response = requests.get("https://www.gutenberg.org/files/1661/1661-0.txt")
sherlock_raw_text = response.text

bpe2 = BytePairEncoder()
bpe2.train(sherlock_raw_text, 3_000)

# leave out first 900 characters of copyright stuff
tokenized_sherlock = bpe2.tokenize(sherlock_raw_text[900:10_000])

In [31]:
print("last few tokens in the vocabulary")
print(bpe2.vocabulary[-20:])

print("tokenizing some text")
# and leave out the table of contents and such
print(tokenized_sherlock[200:500])

last few tokens in the vocabulary
['reck', 'Ġris', 'Ġtrif', 'yal', 'Ġprodu', 'urely', 'clo', 'sive', 'rig', 'ges', 'aff', 'ously', 'Ġmag', 'vering', 'atic', 'Ġ“‘D', 'osed', 'Ġfla', 'Ġpers', 'Ġder']
tokenizing some text
['ĠTo', 'ĠSherlock', 'ĠHolmes', 'Ġshe', 'Ġis', 'Ġalways', 'Ġ', '_', 'the', '_', 'Ġwoman', '.', 'ĠI', 'Ġhave', 'Ġse', 'ld', 'om', 'Ġheard', 'Ġhim', 'Ġm', 'ention', 'Ġher', 'Ġunder', 'Ġany', 'Ġother', 'Ġname', '.', 'ĠIn', 'Ġhis', 'Ġeyes', 'Ġshe', 'Ġe', 'c', 'li', 'ps', 'es', 'Ġand', 'Ġp', 'red', 'om', 'in', 'ates', 'Ġthe', 'Ġwhole', 'Ġof', 'Ġher', 'Ġse', 'x', '.', 'ĠIt', 'Ġwas', 'Ġnot', 'Ġthat', 'Ġhe', 'Ġfelt', 'Ġany', 'Ġem', 'ot', 'ion', 'Ġa', 'k', 'in', 'Ġto', 'Ġlove', 'Ġfor', 'ĠIrene', 'ĠAdler', '.', 'ĠAll', 'Ġem', 'ot', 'ions', ',', 'Ġand', 'Ġthat', 'Ġone', 'Ġparticular', 'ly', ',', 'Ġwere', 'Ġab', 'h', 'or', 're', 'nt', 'Ġto', 'Ġhis', 'Ġcold', ',', 'Ġpre', 'cise', 'Ġbut', 'Ġadmir', 'ably', 'Ġb', 'al', 'anced', 'Ġmind', '.', 'ĠHe', 'Ġwas', ',', 'ĠI', 'Ġtake', 'Ġit', ',